In [ ]:
%pylab inline

In [ ]:
'''
This is a trail to test whether it is approachable to do surface modeling in single brain region.
'''
import time as t0
import numpy
import matplotlib
from tvb.simulator.lab import *
import pandas as pd
from datetime import date
import os
tic = t0.time()

In [ ]:
simulation_mode = 'simu1'
today = date.today().isoformat()
simu_name = today + '_' + simulation_mode
data_folder = os.path.join(os.getcwd(), 'data')

In [ ]:
simulator.Simulator()
# white_matter = connectivity.Connectivity.from_file()
# white_matter.speed = numpy.array([4.0])
# white_matter_coupling = coupling.Linear(a=numpy.array([0.014]))
# specify the coupling
c=coupling.Linear(a=numpy.array([0.5]))
# The algorithm to solve differential equations
# integ = integrators.HeunDeterministic(dt=0.5)
integ = integrators.HeunStochastic(dt=0.5)
# region-based modeling
white_matter = connectivity.Connectivity.from_file()
tract_lengths = white_matter.tract_lengths[73:75,73:75]
region_labels = white_matter.region_labels[73:75]
centres = white_matter.centres[73:75]
cortical = white_matter.cortical[73:75]
orientations = white_matter.orientations[73:75]
areas = white_matter.areas[73:75]
weights = white_matter.weights[73:75,73:75]
two_regions = connectivity.Connectivity(number_of_regions = 2,
                                        number_of_connections=4,
                                        tract_lengths = tract_lengths,
                                        region_labels = region_labels,
                                        centres = centres,
                                        cortical = cortical,
                                        orientations = orientations,
                                        areas = areas,weights = weights)
two_regions.configure()

## Wilson Cowan model
$$\dot{E}_k= \dfrac{1}{\tau_e} (-E_k  + (k_e - r_e E_k) \mathcal{S}_e (\alpha_e \left( c_{ee} E_k - c_{ei} I_k  + P_k - \theta_e + \mathbf{\Gamma}(E_k, E_j, u_{kj}) + W_{\zeta}\cdot E_j + W_{\zeta}\cdot I_j\right) ))\\
\dot{I}_k= \dfrac{1}{\tau_i} (-I_k  + (k_i - r_i I_k) \mathcal{S}_i (\alpha_i \left( c_{ie} E_k - c_{ee} I_k  + Q_k - \theta_i + \mathbf{\Gamma}(E_k, E_j, u_{kj}) + W_{\zeta}\cdot E_j + W_{\zeta}\cdot I_j\right) ))$$

In [ ]:
# Initialize the model
mod = models.WilsonCowan(k_e = np.array([1.0]),            #Max value of excitatory response function
                         k_i = np.array([1.0]),            #Max value of inhibitory response function
                         r_e = np.array([1.0]),            #Excitatory refractory period
                         r_i = np.array([1.0]),            #Inhibitory refractory period
                         c_ee = np.array([16.0]),          #Excitatory to excitatory coupling coefficient
                         c_ei = np.array([15.0]),          #Inhibitory to excitatory coupling coefficient
                         c_ie = np.array([15.11]),         #Excitatory to inhibitory coupling coefficient
                         c_ii = np.array([5.65]),          #Inhibitory to inhibitory coupling coefficient
                         tau_e = np.array([8.0]),         #Membrane time-constant for the excitatory population
                         tau_i = np.array([8.0]),         #Membrane time-constant for the inhibitory population
                         a_e = np.array([1.2]),            #Slope parameter for the excitatory response function
                         b_e = np.array([2.38]),           #Position of the maximum slope of the excitatory sigmoid function
                         c_e = np.array([1.0]),            #Amplitude parameter for the excitatory response function
                         a_i = np.array([1.0]),            #Slope parameter for the inhibitory response function
                         b_i = np.array([2]),            #Position of the maximum slope of the inhibitory sigmoid function
                         c_i = np.array([1.0]),            #Amplitude parameter for the inhibitory response function
                         theta_e = np.array([0.0]),        #Excitatory treshold
                         theta_i = np.array([0.0]),        #Inhibitory treshold
                         alpha_e = np.array([1.0]),        # ?Balance parameter between excitatory and inhibitory masses
                         alpha_i = np.array([1.0]),        # ?Balance parameter between excitatory and inhibitory masses
                         P = np.array([1.8]),              #External stimulus to the excitatory population
                         Q = np.array([0.0]),              #External stimulus to the inhibitory population
                         shift_sigmoid = np.array(True)  # Try False
                        )

In [ ]:
# Use the regional mapping data in the cortex folder
source_path = '/mnt/user/drive/My Libraries/tutorials&explorations/data/cortex/cortex_843.zip'
mapping_path = '/mnt/user/drive/My Libraries/tutorials&explorations/data/cortex/regionMapping_843_2.txt'
local_connectivity_path = '/mnt/user/drive/My Libraries/tutorials&explorations/data/cortex/local_connectivity_843.mat'

In [ ]:
#Initialise a surface
default_cortex = cortex.Cortex.from_file(source_file=source_path, region_mapping_file=mapping_path)
default_cortex.coupling_strength = numpy.array([2**-10])
default_cortex.local_connectivity = local_connectivity.LocalConnectivity.from_file(local_connectivity_path)
default_cortex.region_mapping_data.connectivity = two_regions #white_matter
default_cortex.configure()

In [ ]:
#Initialise some Monitors with period in physical time
mon_tavg = monitors.TemporalAverage(period=0.9765625) # Sampling period: ms; 1024 Hz
mon_savg = monitors.SpatialAverage(period=0.9765625)
# load the default region mapping
# rm = region_mapping.RegionMapping.from_file(source_file=mapping_path)
# mon_eeg = monitors.EEG.from_file() # require whole brain mapping
# mon_eeg.region_mapping=rm
#Bundle them
what_to_watch = (mon_tavg, mon_savg)

In [ ]:
#Initialise Simulator -- Model, Connectivity, Integrator, Monitors, and surface.
sim = simulator.Simulator(model = mod, connectivity = two_regions,
                          coupling = c, 
                          integrator = integ, monitors = (mon_savg,),
                          simulation_length = 50000, #1000000,
                          surface = default_cortex)

sim.configure()

In [ ]:
#Perform the simulation
# tavg_data = []
# tavg_time = []
savg_data = []
savg_time = []
# eeg_data = []
# eeg_time = []
# hf = h5py.File('data.h5','w')
# hf.create_dataset('dts_1', data=d1)
result_name = os.path.join(data_folder,simu_name+"_results.csv")
batch = 0
last_t = []
latt_d = []
for savg in sim():
    # if not tavg is None:
        # tavg_time.append(tavg[0])
        # tavg_data.append(tavg[1])
    if not savg is None:
        savg_time.append(savg[0][0])
        savg_data.append(savg[0][1])
        if len(savg_time) == 200000:
            SAVG = numpy.array(savg_data)
            v1 = SAVG[:,0,0,0]
            v2 = SAVG[:,0,1,0]
            df = pd.DataFrame({'time':savg_time,
                               's_v1':v1,
                              's_v2':v2})
            if batch == 0:
                df.to_csv(result_name, index=False, header=False)
            else:
                df.to_csv(result_name, mode='a', index=False, header=False)
            batch +=1
            last_t = savg_time
            latt_d = savg_data
            df= []
            savg_data = []
            savg_time = []

    # if not eeg is None:
    #     eeg_time.append(eeg[0])
    #     eeg_data.append(eeg[1])

In [ ]:
# result = sim.run()  # another way to run the simulation

In [ ]:
savg_ms = numpy.array(savg_time) #last_t)
savg_ms = savg_ms.astype(float)
time_s = savg_ms/10000

In [ ]:
SAVG = numpy.array(savg_data) #latt_d)

#Plot region averaged time series
fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(111)
ax.plot(savg_time[500:1500], SAVG[500:1500, 0, :, 0])
# ax.plot(time_s[80000:82500], SAVG[80000:82500, 0, :, 0])
title("Region average")
plt.xlabel("time/ms")

show()

In [ ]:
savg_time.shape

In [ ]:
toc = t0.time() - tic
print('Elapsed:',toc)

In [ ]:
# Test the data file
df = pd.read_csv(result_name, header = None)
total_rows=len(df.axes[0])
print("Number of Rows: "+str(total_rows))
df.tail()